## for TA: Instructions on how to import packages

- pip install ta
- pip install pytz

In [1]:
# Imports
import ibapi
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *
import ta
import numpy as np
import pandas as pd
import pytz 
import math
from datetime import datetime, timedelta
import threading
import time

In [2]:
class IBApi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self,self)
        
        #used for caching historical data for placing dummy trades outside of market hours
        self.data = [] 
        
    #historical backtest data
    def historicalData(self, reqId, bar):
        #cache historical data
        self.data.append([bar.date, bar.high, bar.low, bar.close])
            
    #on historical data end
    def historicalDataEnd(self, reqId, start, end):
        print(reqId)
    
    #get next order id we can use
    def nextValidId(self, nextorderId):
        global orderId
        orderId = nextorderId
            
    def error(self, id, errorCode, errorMsg):
        print("TWS Error Code: ", errorCode)
        print("Message for Error Code: ",errorMsg)

In [3]:
# Bot Logic
class Bot:
    
    ib = None
    testIb = None
    bars = []
    reqId = 1
    global orderId
    testOrderId = 1000 #increment everyday, resets at midnight
    smaPeriod = 50
    symbol = ''
    
    def __init__(self):
        
        #connect to IB on init
        self.ib = IBApi()
        
        self.ib.connect('127.0.0.1', 7497, 126)
        ib_thread = threading.Thread(target=self.run_loop, daemon=True)
        
        ib_thread.start()
        time.sleep(1)
        
        # get inputs
        self.symbol = input('Enter the ticker you want to trade: ')
        self.barsize = input('Enter the bar size: ')
        
        minQuantity = ' min'
        if (int(self.barsize) > 1):
            minQuantity = ' mins'
        
        #create contract object
        contract = Contract()
        contract.symbol = self.symbol.upper() 
        contract.secType = 'STK'
        contract.exchange = 'SMART'
        contract.currency = 'USD'
        self.ib.reqIds(-1)
        
        #request market data
        print("Requesting market data...")
        self.ib.reqHistoricalData(self.reqId, contract, '', '2 D', str(self.barsize)+minQuantity, 'TRADES',1,1,True,[])
        
        # for testing trading strategy outside of market hours
        testHistData = self.getHistoricalData()
        self.testTradingStrategy(testHistData)
    
    #listen to socket in seperate thread
    def run_loop(self):
        self.ib.run()
    
    # NOTE: to only be used outside of trading hours - trading strategy logic test
    # should only be used for placing dummy trades
    def testTradingStrategy(self,testDf):
        
        # reverse dataframe to ensure order is earliest to oldest
        testDf = testDf.iloc[::-1]
        testCloses = testDf['Test: Close']
        self.testSMA = ta.trend.sma_indicator(testCloses, 50, True)
        
        #start from the end (earliest) and work our way up(latest)
        print(testDf.head())
        
        lastLow = testDf.iloc[0,1]
        lastHigh = testDf.iloc[0,0]
        lastClose = testDf.iloc[0,2]
        lastBar = testDf.iloc[[0]]
        
        topHigh = None
        topLow = None
        
        counter = 1
        currentBar = testDf.iloc[counter]
        
        
        #check criteria
        while (counter < len(testDf) - 1):
            
            if (currentBar['Test: Close'] > lastHigh
                and currentBar['Test: Low'] > lastLow
                and currentBar['Test: Close'] > self.testSMA[len(self.testSMA)-1]
                and lastClose < self.testSMA[len(self.testSMA)-1]):
                
                print("counter is at: ",counter)

                #we leave when we either have a 1% loss or 2% gain
                profitTarget = currentBar['Test: Close']*1.02
                stopLoss = currentBar['Test: Close']*0.99
                quantity = 50
                bracket = self.bracketOrder(self.testOrderId, 'BUY',quantity, profitTarget, stopLoss)
                contract = Contract()
                contract.symbol = self.symbol.upper() 
                contract.secType = 'STK'
                contract.exchange = 'SMART'
                contract.currency = 'USD'
                
                #place bracket order
                for x in bracket:
                    x.ocaGroup = 'OCA_'+str(self.testOrderId)
                    x.ocaType = 2
                    orderId = self.testOrderId
                    self.ib.placeOrder(x.orderId, contract, x)
                    
                self.testOrderId += 3


            counter += 1 
            currentBar = testDf.iloc[counter]
        
    # NOTE: to only be used outside of trading hours
    # should only be used for placing dummy trades
    def getHistoricalData(self):
        
        def run_loop():
            self.testIb.run()
        
        self.testIb = IBApi()
        self.testIb.connect('127.0.0.1', 7497, 16)

        #Start the socket in a thread
        api_thread = threading.Thread(target=run_loop, daemon=True)
        api_thread.start()

        time.sleep(1) 

        #Create contract object
        testContract = Contract()
        testContract.symbol = self.symbol
        testContract.secType = 'STK'
        testContract.exchange = 'SMART'
        testContract.currency = 'USD'
        testContract.primaryExchange = 'NASDAQ'
        
        #Request Trade Data
        self.testIb.reqHistoricalData(8, testContract,'', '1 D', '1 min', 'TRADES', 0, 1, False, [])

        recievedData = self.testIb.data
        
        time.sleep(5)
        self.testIb.disconnect()
        
        testDf = pd.DataFrame(recievedData, columns=['Date', 'Test: High', 'Test: Low', 'Test: Close',])
        testDf = testDf.set_index('Date')
        
        return testDf
        
    #bracket order
    def bracketOrder(self, parentOrderId, action, quantity, profitTarget, stopLoss):
        
        #initial entry
        contract = Contract()
        contract.symbol = self.symbol.upper() 
        contract.secType = 'STK'
        contract.exchange = 'SMART'
        contract.currency = 'USD'
        
        #create parent order 
        parent = Order()
        parent.orderId = parentOrderId
        parent.orderType = 'MKT'
        parent.action = action
        parent.totalQuantity = quantity
        parent.transmit = False 
        
        # make a target order
        profitOrder = Order()
        profitOrder.orderId = parent.orderId+1
        profitOrder.orderType = 'LMT'
        profitOrder.action = 'SELL'
        profitOrder.totalQuantity = quantity
        profitOrder.lmtPrice = round(profitTarget,3)
        profitOrder.transmit = False 
        
        # make a loss order
        stopOrder = Order()
        stopOrder.orderId = parent.orderId+2
        stopOrder.orderType = 'STP'
        stopOrder.action = 'SELL'
        stopOrder.totalQuantity = quantity
        stopOrder.parentId = parentOrderId 
        stopOrder.auxPrice = round(stopLoss,3)
        stopOrder.transmit = True 
        
        return [parent, profitOrder, stopOrder]
                

In [4]:
####################### Start Bot #########################
bot = Bot()

TWS Error Code:  2104
Message for Error Code:  Market data farm connection is OK:usfarm.nj
TWS Error Code:  2104
Message for Error Code:  Market data farm connection is OK:cashfarm
TWS Error Code:  2104
Message for Error Code:  Market data farm connection is OK:usfarm
TWS Error Code:  2106
Message for Error Code:  HMDS data farm connection is OK:ushmds
TWS Error Code:  2158
Message for Error Code:  Sec-def data farm connection is OK:secdefil
Enter the ticker you want to trade: NIO
Enter the bar size: 1
Requesting market data...
TWS Error Code:  2104
Message for Error Code:  Market data farm connection is OK:usfarm.nj
TWS Error Code:  2104
Message for Error Code:  Market data farm connection is OK:cashfarm
TWS Error Code:  2104
Message for Error Code:  Market data farm connection is OK:usfarm
TWS Error Code:  2106
Message for Error Code:  HMDS data farm connection is OK:ushmds
TWS Error Code:  2158
Message for Error Code:  Sec-def data farm connection is OK:secdefil
1
8
                

TWS Error Code:  399
Message for Error Code:  Order Message: BUY 50 NIO NYSE Warning: your order will not be placed at the exchange until 2022-09-23 09:30:00 US/Eastern
TWS Error Code:  2168
Message for Error Code:  Warning: The 'EtradeOnly' order attribute is not supported.
TWS Error Code:  2169
Message for Error Code:  Warning: The 'FirmQuoteOnly' order attribute is not supported.
TWS Error Code:  2168
Message for Error Code:  Warning: The 'EtradeOnly' order attribute is not supported.
TWS Error Code:  2169
Message for Error Code:  Warning: The 'FirmQuoteOnly' order attribute is not supported.
TWS Error Code:  2168
Message for Error Code:  Warning: The 'EtradeOnly' order attribute is not supported.
TWS Error Code:  2169
Message for Error Code:  Warning: The 'FirmQuoteOnly' order attribute is not supported.
TWS Error Code:  2168
Message for Error Code:  Warning: The 'EtradeOnly' order attribute is not supported.
TWS Error Code:  2169
Message for Error Code:  Warning: The 'FirmQuoteOnl

TWS Error Code:  399
Message for Error Code:  Order Message: BUY 50 NIO NYSE Warning: your order will not be placed at the exchange until 2022-09-23 09:30:00 US/Eastern
TWS Error Code:  399
Message for Error Code:  Order Message: SELL 50 NIO NYSE Warning: your order will not be placed at the exchange until 2022-09-23 09:30:00 US/Eastern
TWS Error Code:  399
Message for Error Code:  Order Message: BUY 50 NIO NYSE Warning: your order will not be placed at the exchange until 2022-09-23 09:30:00 US/Eastern
TWS Error Code:  399
Message for Error Code:  Order Message: SELL 50 NIO NYSE Warning: your order will not be placed at the exchange until 2022-09-23 09:30:00 US/Eastern
TWS Error Code:  399
Message for Error Code:  Order Message: BUY 50 NIO NYSE Warning: your order will not be placed at the exchange until 2022-09-23 09:30:00 US/Eastern
TWS Error Code:  399
Message for Error Code:  Order Message: SELL 50 NIO NYSE Warning: your order will not be placed at the exchange until 2022-09-23 09: